<a href="https://colab.research.google.com/github/how-to-train-your-model/data-science-portfolio/blob/main/Chatbot_Foodie_Development_PP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot to find healthy recipes and notritional facts

What's On Your Plate is a platform that collects the recipes from restaurants all over the world , allowing their customers to view the ingredients, nutrition facts and preparation methods for more than 50,000 recipes.

They conducted a survey on customers interested tin the restaurant menu. People wanted to know all the ingredients and the nutritional facts of the food, so we decided to provide answers to customers’ questions about their food.

Recipe 1M+ is a large-scale, structured corpus of over one million cooking recipes and 13 million food images.

Today, I am introducing the chatbot “Foodie”. Foodie will help you find food based on your cravings, food allergies and dietary restrictions.
The bot will respond to questions regarding different recipes, preparation directions, and nutrition facts.

## Importing Packages

In [1]:
import re
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import seaborn as sns

import os
import re
import string
from packaging import version


from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
#from rake_nltk import Rake

import re,string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import ngrams, FreqDist
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pickle
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

import datetime
from packaging import version
from collections import Counter
import time
from timeit import default_timer as timer #import a timer
from datetime import datetime



from numpy import array
from numpy import argmax

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import ngrams, FreqDist

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, silhouette_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.manifold import TSNE

import gensim
from gensim.models import Word2Vec,LdaMulticore, TfidfModel
from gensim import corpora
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
# Only run this once, they will be downloaded.
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Mount Google Drive to Colab Environment

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [6]:
# Allows for muliple output in one cell window
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Allows for viewing the complete data in a single window 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

<div class="alert alert-block alert-info">
    <b>Spacy </b><br>
    https://spacy.io/usage/models
    </div>

In [7]:
nlp = spacy.load('en_core_web_sm')

## Data processing

In [8]:
#stop_words = set(stopwords.words('english'))
stopwords = nltk.corpus.stopwords.words('english')


In [9]:
 stopwords = nltk.corpus.stopwords.words('english')
# newStopWords = ['g'. 're']
# stopwords.extend(newStopWords)

In [10]:
# #Removing duplicate stop words
# def Remove(duplicate):
#     final_list = []
#     for num in duplicate:
#         if num not in final_list:
#             final_list.append(num)
#     return final_list
     
# stop_words_final = Remove(stopwords)
# print(stop_words_final)

In [11]:
def remove_stop_words(in_text):
    #stop_words = set(stopwords.words('english')) 
    stop_words = set(stopwords)
    word_tokens = word_tokenize(in_text)  
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 

    return filtered_sentence

### Clean doc function

In [12]:
def clean_doc(doc): 
    #split document into individual words
    doc = ' '.join(remove_stop_words(doc))
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 4]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    #tokens = [w for w in tokens if not w in stop_words]  
       

    return tokens    

### Load Data

In [13]:
!ls -l "/content/drive/My Drive/Colab_Notebooks/MSDS-453/Final_Project"

total 223398
-rw------- 1 root root    216479 Mar  1 22:22 kg_data_cleanup.csv
-rw------- 1 root root     15255 Mar  1 22:52 kg_data_cleanup_done.csv
-rw------- 1 root root       143 Mar  1 22:52 kg_data_cleanup.gsheet
-rw------- 1 root root    701048 Mar  2 16:43 Recipe_Book_final.csv
-rw------- 1 root root       143 Mar  8 22:07 Recipe_Book_final.gsheet
-rw------- 1 root root       143 Mar  1 05:51 Recipe_Book.gsheet
-rw------- 1 root root       143 Mar  2 16:33 Recipe_Book_v1.gsheet
-rw------- 1 root root   5127867 Mar 10 00:00 Recipe_Chatbot_Data_Analysis.ipynb
-rw------- 1 root root    158099 Mar 10 00:22 Recipe_Chatbot_Model_Development.ipynb
-rw------- 1 root root 222537053 Mar  1 01:34 recipes_with_nutritional_info.json


In [14]:
path_to_file=('/content/drive/My Drive/Colab_Notebooks/MSDS-453/Final_Project/Recipe_Book_final.csv')
data = pd.read_csv(path_to_file)
data.head(20)



,Doc_ID,Meal_ID,Ingredients_Directions
0,0,Fluffiest_Blueberry_Pancakes,Super thick and fluffy blueberry pancakes! Mel...
1,1,The_Best_Green_Smoothie,A healthy breakfast - The Best Green Smoothie\...
2,2,Lemon_Poppyseed_Zucchini_Bread,"Lemon Poppyseed Zucchini Bread with zucchini, ..."
3,3,Spicy_Shrimp_Tacos_with_Garlic_Cilantro_Lime_Slaw,The BEST Shrimp Tacos with Garlic Cilantro Lim...
4,4,Chipotle_Tahini_Bowls,Healthy and versatile bowls that come together...
5,5,Sheet_Pan_Chicken_Pitas_with_Tzatziki,Sheet Pan Chicken Pitas that are salty-briney-...
6,6,The_Best_Soft_Chocolate_Chip_Cookies,These are THE BEST soft chocolate chip cookies...
7,7,Raspberry_Crumble_Bars,These Raspberry Crumble Bars are one of the be...
8,8,Best_Peach_Cobbler,I have tried so many peach cobbler recipes and...
9,9,Yogurt_Parfaits,"'yogurt, greek, plain, nonfat', 'strawberrie..."


In [15]:
#adding two columns to the dataframe to store the processed text and tokenized text
data['processed_text'] = data['Ingredients_Directions'].apply(lambda x: clean_doc(x))

recipe_data = data.copy()
recipe_data.head(2)

,Doc_ID,Meal_ID,Ingredients_Directions,processed_text
0,0,Fluffiest_Blueberry_Pancakes,Super thick and fluffy blueberry pancakes! Mel...,"[super, thick, fluffy, blueberry, pancakes, mo..."
1,1,The_Best_Green_Smoothie,A healthy breakfast - The Best Green Smoothie\...,"[healthy, breakfast, green, smoothie, reviews,..."


In [16]:
data.head()
data.shape
data['Meal_ID'].head().unique()

,Doc_ID,Meal_ID,Ingredients_Directions,processed_text
0,0,Fluffiest_Blueberry_Pancakes,Super thick and fluffy blueberry pancakes! Mel...,"[super, thick, fluffy, blueberry, pancakes, mo..."
1,1,The_Best_Green_Smoothie,A healthy breakfast - The Best Green Smoothie\...,"[healthy, breakfast, green, smoothie, reviews,..."
2,2,Lemon_Poppyseed_Zucchini_Bread,"Lemon Poppyseed Zucchini Bread with zucchini, ...","[lemon, poppyseed, zucchini, bread, zucchini, ..."
3,3,Spicy_Shrimp_Tacos_with_Garlic_Cilantro_Lime_Slaw,The BEST Shrimp Tacos with Garlic Cilantro Lim...,"[shrimp, tacos, garlic, cilantro, ready, minut..."
4,4,Chipotle_Tahini_Bowls,Healthy and versatile bowls that come together...,"[healthy, versatile, bowls, together, generous..."


(501, 4)

array(['Fluffiest_Blueberry_Pancakes', 'The_Best_Green_Smoothie',
       'Lemon_Poppyseed_Zucchini_Bread',
       'Spicy_Shrimp_Tacos_with_Garlic_Cilantro_Lime_Slaw',
       'Chipotle_Tahini_Bowls'], dtype=object)

## Chatbot Data prep

In [17]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# Allows for muliple output in one cell window
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Allows for viewing the complete data in a single window 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!ls -l '/content/drive/My Drive/Colab_Notebooks/MSDS-453/Final_Project'

total 223398
-rw------- 1 root root    216479 Mar  1 22:22 kg_data_cleanup.csv
-rw------- 1 root root     15255 Mar  1 22:52 kg_data_cleanup_done.csv
-rw------- 1 root root       143 Mar  1 22:52 kg_data_cleanup.gsheet
-rw------- 1 root root    701048 Mar  2 16:43 Recipe_Book_final.csv
-rw------- 1 root root       143 Mar  8 22:07 Recipe_Book_final.gsheet
-rw------- 1 root root       143 Mar  1 05:51 Recipe_Book.gsheet
-rw------- 1 root root       143 Mar  2 16:33 Recipe_Book_v1.gsheet
-rw------- 1 root root   5127867 Mar 10 00:00 Recipe_Chatbot_Data_Analysis.ipynb
-rw------- 1 root root    158099 Mar 10 00:22 Recipe_Chatbot_Model_Development.ipynb
-rw------- 1 root root 222537053 Mar  1 01:34 recipes_with_nutritional_info.json


In [19]:
path_to_file=('/content/drive/My Drive/Colab_Notebooks/MSDS-453/Final_Project/Recipe_Book_final.csv')
data1 = pd.read_csv(path_to_file)
data1.head(20)



,Doc_ID,Meal_ID,Ingredients_Directions
0,0,Fluffiest_Blueberry_Pancakes,Super thick and fluffy blueberry pancakes! Mel...
1,1,The_Best_Green_Smoothie,A healthy breakfast - The Best Green Smoothie\...
2,2,Lemon_Poppyseed_Zucchini_Bread,"Lemon Poppyseed Zucchini Bread with zucchini, ..."
3,3,Spicy_Shrimp_Tacos_with_Garlic_Cilantro_Lime_Slaw,The BEST Shrimp Tacos with Garlic Cilantro Lim...
4,4,Chipotle_Tahini_Bowls,Healthy and versatile bowls that come together...
5,5,Sheet_Pan_Chicken_Pitas_with_Tzatziki,Sheet Pan Chicken Pitas that are salty-briney-...
6,6,The_Best_Soft_Chocolate_Chip_Cookies,These are THE BEST soft chocolate chip cookies...
7,7,Raspberry_Crumble_Bars,These Raspberry Crumble Bars are one of the be...
8,8,Best_Peach_Cobbler,I have tried so many peach cobbler recipes and...
9,9,Yogurt_Parfaits,"'yogurt, greek, plain, nonfat', 'strawberrie..."


In [20]:
recipe_data1 = data1.copy()
recipe_data1.head(10)
recipe_data1.shape

,Doc_ID,Meal_ID,Ingredients_Directions
0,0,Fluffiest_Blueberry_Pancakes,Super thick and fluffy blueberry pancakes! Mel...
1,1,The_Best_Green_Smoothie,A healthy breakfast - The Best Green Smoothie\...
2,2,Lemon_Poppyseed_Zucchini_Bread,"Lemon Poppyseed Zucchini Bread with zucchini, ..."
3,3,Spicy_Shrimp_Tacos_with_Garlic_Cilantro_Lime_Slaw,The BEST Shrimp Tacos with Garlic Cilantro Lim...
4,4,Chipotle_Tahini_Bowls,Healthy and versatile bowls that come together...
5,5,Sheet_Pan_Chicken_Pitas_with_Tzatziki,Sheet Pan Chicken Pitas that are salty-briney-...
6,6,The_Best_Soft_Chocolate_Chip_Cookies,These are THE BEST soft chocolate chip cookies...
7,7,Raspberry_Crumble_Bars,These Raspberry Crumble Bars are one of the be...
8,8,Best_Peach_Cobbler,I have tried so many peach cobbler recipes and...
9,9,Yogurt_Parfaits,"'yogurt, greek, plain, nonfat', 'strawberrie..."


(501, 3)

In [21]:
recipe_data1['Ingredients_Directions'][0]


'Super thick and fluffy blueberry pancakes! Melt in your mouth, golden brown, and bursting with blueberries for breakfast.\nIngredients\nScale\n\n3/4 cup milk\n2 tablespoons white vinegar\n1 cup flour\n2 tablespoons sugar\n1 teaspoon baking powder\n1/2 teaspoon baking soda\n1/2 teaspoon salt\n1 egg\n2 tablespoons melted butter\n1+ cup fresh blueberries\nmore butter for the pan\n\nInstructions\n\nMix the milk and vinegar and let it sit for a minute or two (you’re making “buttermilk” here).\nWhisk the dry ingredients together. Whisk the egg, milk, and melted butter into the dry ingredients until just combined.\nHeat a nonstick pan over medium heat. Melt a little smear of butter in the pan (essential for giving a yummy golden brown crust).\nPour about 1/3 cup of batter into the hot skillet and spread it flat-like (it will be pretty thick). Arrange a few blueberries on top. Cook until you see little bubbles on top and the edges starting to firm up. Flip and cook for another 1-2 minutes unt

In [22]:
word_tokens1 = []
for i in range(0,len(recipe_data1)):
  word_tokens = nltk.word_tokenize(recipe_data1['Ingredients_Directions'][i])
  word_tokens1.append(nltk.word_tokenize(recipe_data1['Ingredients_Directions'][i]))
#word_tokens[:1]
#word_tokens1[:2]
type(word_tokens)
#type(word_tokens1)
#temp_data = [line.strip() for line in word_tokens1]
#word_tokens2 = [[word.lower() for word in text.split()] for text in temp_data]
#word_tokens2[:1]

#type(word_tokens2)


list

In [23]:
sent_tokens1 = []
for i in range(0,len(recipe_data1)):
  #sent_tokens = nltk.sent_tokenize(recipe_data1['Ingredients_Directions'][i])
  sent_tokens1.append(nltk.sent_tokenize(recipe_data1['Ingredients_Directions'][i]))
sent_tokens1[:2]
type(sent_tokens1)

[['Super thick and fluffy blueberry pancakes!',
  'Melt in your mouth, golden brown, and bursting with blueberries for breakfast.',
  'Ingredients\nScale\n\n3/4 cup milk\n2 tablespoons white vinegar\n1 cup flour\n2 tablespoons sugar\n1 teaspoon baking powder\n1/2 teaspoon baking soda\n1/2 teaspoon salt\n1 egg\n2 tablespoons melted butter\n1+ cup fresh blueberries\nmore butter for the pan\n\nInstructions\n\nMix the milk and vinegar and let it sit for a minute or two (you’re making “buttermilk” here).',
  'Whisk the dry ingredients together.',
  'Whisk the egg, milk, and melted butter into the dry ingredients until just combined.',
  'Heat a nonstick pan over medium heat.',
  'Melt a little smear of butter in the pan (essential for giving a yummy golden brown crust).',
  'Pour about 1/3 cup of batter into the hot skillet and spread it flat-like (it will be pretty thick).',
  'Arrange a few blueberries on top.',
  'Cook until you see little bubbles on top and the edges starting to firm up

list

### Pre-processing the raw text

In [24]:
from collections import Iterable
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

word_tokens1 = list(flatten(word_tokens1))
sent_tokens1 = list(flatten(sent_tokens1))

sent_tokens1[:2]



['Super thick and fluffy blueberry pancakes!',
 'Melt in your mouth, golden brown, and bursting with blueberries for breakfast.']

In [25]:
def clean_doc(doc): 
    #split document into individual words
    #doc = ' '.join(remove_stop_words(doc))
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 4]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    #tokens = [w for w in tokens if not w in stop_words]  
       

    return tokens    

In [26]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    #return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
    #return LemTokens(nltk.word_tokenize(text.translate(remove_punct_dict)))
    return clean_doc(text)




### Keyword matching

In [27]:
import random
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey", "I am doing well")
GREETING_RESPONSES = ["hi - how are you?", "hey - how are you", "hola - how are you", "hi there - how are you", "hello - how are you", "I am glad! You are talking to me", "How can I help"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [28]:
def response(user_response):
    Foodie_response=''
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens1)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        Foodie_response=Foodie_response+"I am sorry! I don't understand you"
        return Foodie_response
    else:
        Foodie_response = Foodie_response+sent_tokens1[idx]
        return Foodie_response

In [29]:
flag=True
print("Foodie: Welcome to What's on your Plate, Foodie at your service!\n If you are here by mistake, type Bye and I will see you soon!")
while(flag==True):
    user_response = input()
    #user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Foodie: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Foodie: "+greeting(user_response))
            else:
                sent_tokens1.append(user_response)
                word_tokens1=word_tokens1+nltk.word_tokenize(user_response)
                final_words=word_tokens1
                #final_words=list(set(word_tokens1))
                print("Foodie: ",end="")
                print(response(user_response))
                sent_tokens1.remove(user_response)
    else:
        flag=False
        print("Foodie: Bye! take care..")

Foodie: Welcome to What's on your Plate, Foodie at your service!
 If you are here by mistake, type Bye and I will see you soon!
hello
Foodie: I am glad! You are talking to me
ginger
Foodie: ',  'If peaches are very ripe, add 1/2 cup ginger ale; otherwise, add 1 cup.
bye
Foodie: Bye! take care..


## ChatBot using BERT (didn't work)

In [30]:
#!pip install transformers
!pip install pytorch_lightning

In [31]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import LongformerConfig, LongformerModel, LongformerTokenizer

import pytorch_lightning as pl

In [32]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# Allows for muliple output in one cell window
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Allows for viewing the complete data in a single window 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# Initializing a Longformer configuration
configuration = LongformerConfig()

# Initializing a model from the configuration
model = LongformerModel(configuration)

# Accessing the model configuration
configuration = model.config

In [48]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [36]:
#model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
#tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    answer = ""
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b    
    
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)    
    
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer:\n{}".format(answer.capitalize()))

In [50]:
recipe_data1.head()

,Doc_ID,Meal_ID,Ingredients_Directions
0,0,Fluffiest_Blueberry_Pancakes,Super thick and fluffy blueberry pancakes! Mel...
1,1,The_Best_Green_Smoothie,A healthy breakfast - The Best Green Smoothie\...
2,2,Lemon_Poppyseed_Zucchini_Bread,"Lemon Poppyseed Zucchini Bread with zucchini, ..."
3,3,Spicy_Shrimp_Tacos_with_Garlic_Cilantro_Lime_Slaw,The BEST Shrimp Tacos with Garlic Cilantro Lim...
4,4,Chipotle_Tahini_Bowls,Healthy and versatile bowls that come together...


In [51]:
text = ' '.join([x for x in recipe_data1['Ingredients_Directions']]).split('.')


In [52]:
whole_doc = ""
recipe_data2 = recipe_data1.head(2).reset_index()  
# make sure indexes pair with number of rows
for index, row in recipe_data2.iterrows():
  whole_doc1 = whole_doc.join(recipe_data2['Ingredients_Directions'][index])
    
    
#print(index)
type(whole_doc1)




str

In [53]:

print('string length:',len(whole_doc1))
a = recipe_data2['Ingredients_Directions'][1]
a


string length: 1731


'A healthy breakfast - The Best Green Smoothie\n \n4.8 from 17 reviews\n\nThe best basic, smooth, creamy green smoothie. So easy – just blend peaches, mango, kale, almond milk, and ginger. Honey or cinnamon if you want, too! _\nIngredients\nUnits\nScale\n\n1 cup frozen mango chunks\n1/2 cup frozen peach slices\na handful of kale (stems removed) or spinach\n1–2 cups almond milk\n1/2 inch slice of fresh ginger\nsprinkle of cinnamon\nhoney to taste\n\nInstructions\n\nBlend until smooth!\nEnjoy!\n\nNotes\n\nAdd more or less almond milk to make this more like a smoothie vs. more like a juice. You can also use water instead of almond milk if you prefer. Coconut water is also nice although it has a bit more natural sugar. Sometimes I find adding a tiny pinch of salt helps the flavor a little bit. This works best with frozen fruit so you get a nice smooth and creamy texture. Protein powder would be a great addition. I’ve also added coconut oil and/or cashews for an extra fat/protein boost. You

In [54]:


question = "Anything with a hint of ginger"

In [55]:
question_answer(question, whole_doc1)

#original answer from the dataset
#print("Original answer:\n", data.loc[data["question"] == question]["answer"].values[0])



Predicted answer:
Unable to find the answer to your question.
